In [43]:
import spectral.io.envi as envi
from proj.tasks import send_matrix_to_queue_task, read_image_spectral, clf_RF_1, read_col_data_from_file, load_sklearn_model_to_file, get_full_path
import numpy as np
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import json
from datetime import datetime
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colormaps
import time 
import dill

In [44]:
# Load object file to code
def load_sklearn_model_to_file_by_cloudpickle(file_path):
    with open(file_path, 'rb') as f:
        return dill.load(f)

In [45]:
# Load sk-learn model
model = load_sklearn_model_to_file_by_cloudpickle(get_full_path("../../model_saved/DT_save/J_model_predict_K_DT_24_02_2024_cloudpickle.pkl"))

In [46]:
# Read spectral image
img = read_image_spectral()

In [47]:
# Get 
NUM_ROW, NUM_COL, NUM_BAND = img.shape
BANDS_ZERO = np.zeros((122), dtype=np.uint8)

In [48]:
NUM_ROW

6478

In [49]:
NUM_COL

5287

In [50]:
def col_data(col):
    file_path = f"./data/img_col_data/img_{col}.npz"
    return read_col_data_from_file(file_path)

In [51]:
def get_position_of_bands_zero(col_data):
    matching_rows = np.where(np.all(col_data == BANDS_ZERO, axis=1))
    return matching_rows[0]

In [52]:

def predict_target_value_for_img(saved_result_path):
    if os.path.exists(saved_result_path):
        print(f"File {saved_result_path} already exists.")
        with np.load(saved_result_path) as file:
            return file["arr_0"]
    
    print(f"File {saved_result_path} NOT exists.")
    target_value = np.zeros((NUM_ROW, NUM_COL))
    for col in range(NUM_COL):
        if col % 100 == 0:
            print(col)
        file_path = f"./data/img_col_data/img_{col}.npz"
        bands = read_col_data_from_file(file_path)
        bands_ix = model.bands_ix
        target_value[:, col] = model.predict(bands[:, bands_ix])
        band_zeros_position = get_position_of_bands_zero(bands)
        target_value[band_zeros_position, col] = 0
    np.savez_compressed(saved_result_path, target_value)
    return target_value

In [53]:
target_values = predict_target_value_for_img("./data/saved_result/J_model_predict_K_DT_24_02_2024.npz")

File ./data/saved_result/J_model_predict_K_DT_24_02_2024.npz NOT exists.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200


In [54]:
target_values.shape

(6478, 5287)